In [1]:
import torch
import malt

dummy_targets = torch.Tensor([0.0])

regressor = malt.models.regressor.GPyTorchExactRegressor(
    dummy_targets,
    in_features=128,
    out_features=2,
)

Using backend: pytorch


In [2]:
import torch
import dgl
import malt

dummy_targets = torch.Tensor([0.0])

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.GPyTorchExactRegressor(
        dummy_targets,
        in_features=128,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

point = malt.Molecule("C")
point.featurize()
graph = dgl.batch([point.g])

y = net.condition(graph)
assert y.mean.item() == 0.0

In [3]:
import torch
import dgl
import malt
import gpytorch

dummy_targets = torch.tensor([[4.0]])

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=32
    ),
    regressor=malt.models.regressor.GPyTorchExactRegressor(
        dummy_targets,
        in_features=32,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

point = malt.Molecule("CCCCCCC")
point.featurize()
graph = dgl.batch([point.g])

mll = gpytorch.mlls.ExactMarginalLogLikelihood(net.regressor.likelihood, net)

net.train()
y_pred = net(graph)
loss = mll(y_pred, torch.tensor([[2.0]]))
loss.backward()

net.eval()
y = net(graph)
assert y.mean.item() != 0.0

In [5]:
import torch
import dgl
import malt

dataset = malt.data.collections.linear_alkanes(10)
dataset_loader = dataset.view(batch_size=len(dataset))
g, y = next(iter(dataset_loader))

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.GPyTorchExactRegressor(
        y,
        in_features=128,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

if torch.cuda.is_available():
    net.cuda()

mll = gpytorch.mlls.ExactMarginalLogLikelihood(net.regressor.likelihood, net)

net.train()
y_pred = net(g)
loss = mll(y_pred, y).mean()
loss.backward()

net.eval()
y_hat = net(g)
assert y_hat.mean.shape[0] == 10
assert len(y_hat.mean.shape) == 1

In [13]:
import malt
import torch
from malt.agents.player import SequentialModelBasedPlayer

dataset = malt.data.collections.linear_alkanes(10)
g, y = dataset.batch()
model = malt.models.supervised_model.GaussianProcessSupervisedModel(
   representation=malt.models.representation.DGLRepresentation(
       out_features=32
   ),
   regressor=malt.models.regressor.ExactGaussianProcessRegressor(
       y, in_features=32, out_features=2,
   ),
   likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)
if torch.cuda.is_available():
    model.cuda()
    
mll = malt.models.marginal_likelihood.ExactMarginalLogLikelihood(
    model.regressor.likelihood,
    model
)

player = SequentialModelBasedPlayer(
    model = model,
    policy=malt.policy.Greedy(),
    trainer=malt.trainer.get_default_trainer(),
    marginal_likelihood=mll,
    merchant=malt.agents.merchant.DatasetMerchant(dataset),
    assayer=malt.agents.assayer.DatasetAssayer(dataset),
)

while True:
    if player.step() is None:
        break

In [40]:
import torch
import dgl
import malt

dataset = malt.data.collections.linear_alkanes(10)
dataset_loader = dataset.view(batch_size=len(dataset))
g, y = next(iter(dataset_loader))
y = y.reshape(-1, 1)


net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.HardcodedExactGaussianProcessRegressor(
        y,
        in_features=128,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

if torch.cuda.is_available():
    net.cuda()


mll = malt.models.marginal_likelihood.HardcodedExactMarginalLogLikelihood(
    net.likelihood, net
)

net.train()
y_hat = net(g)
loss = mll(y_hat, y)
loss.backward()

y_hat = net.condition(g)
assert y_hat.mean.shape[0] == 10
assert len(y_hat.mean.shape) == 1

In [41]:
dataset = malt.data.collections.esol()
dataset.shuffle()
ds_tr, ds_te = dataset[:800], dataset[800:]

Processing dgl graphs from scratch...
Processing molecule 1000/1128


In [47]:
g, y = ds_tr.batch()

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=60
    ),
    regressor=malt.models.regressor.HardcodedExactGaussianProcessRegressor(
        y,
        in_features=60,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

if torch.cuda.is_available():
    net.cuda()


mll = malt.models.marginal_likelihood.HardcodedExactMarginalLogLikelihood(
    net.likelihood, net
)

y_hat = net(g)
loss = mll(y_hat, y)
loss.backward()

y_hat = net.condition(g)

g, y = ds_te.batch()

net.eval()
with torch.no_grad():
    net(g)

In [33]:
# g, y = next(iter(dataset[:4].view(batch_size=4)))
# net(g)

In [14]:
import torch
import dgl
import malt

dummy_targets = torch.Tensor([4.0]*9)

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.HardcodedExactGaussianProcessRegressor(
        dummy_targets,
        in_features=128,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

point = malt.Molecule("C")
point.featurize()
graph = dgl.batch([point.g, point.g])

mll = malt.models.marginal_likelihood.HardcodedExactMarginalLogLikelihood(
    net.likelihood, net
)

net.train()
y_hat = net(graph)
loss = mll(y_hat, torch.tensor([[5.0, 5.0]]))
loss.backward()

net.eval()
y = net.condition(graph)
assert y.mean.item() != 0.0

RuntimeError: Incompatible matrix sizes for triangular_solve: each A matrix is 2 by 2 but each b matrix is 9 by 1

In [12]:
net.condition(graph)

MultivariateNormal(loc: tensor([3.8103], grad_fn=<ExpandBackward0>), covariance_matrix: tensor([[0.0474]], grad_fn=<ExpandBackward0>))

In [11]:
%load_ext autoreload
%autoreload 2

import malt
import torch
from malt.agents.player import SequentialModelBasedPlayer

dataset = malt.data.collections.linear_alkanes(10)
g, y = dataset.batch()
model = malt.models.supervised_model.GaussianProcessSupervisedModel(
   representation=malt.models.representation.DGLRepresentation(
       out_features=32
   ),
   regressor=malt.models.regressor.ExactGaussianProcessRegressor(
       y, in_features=32, out_features=2,
   ),
   likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)
if torch.cuda.is_available():
    model.cuda()

mll = malt.models.marginal_likelihood.ExactMarginalLogLikelihood(
    model.regressor.likelihood,
    model
)

player = SequentialModelBasedPlayer(
    model = model,
    policy=malt.policy.Greedy(),
    trainer=malt.trainer.get_default_trainer(),
    marginal_likelihood=mll,
    merchant=malt.agents.merchant.DatasetMerchant(dataset),
    assayer=malt.agents.assayer.DatasetAssayer(dataset),
)

while True:
    if player.step() is None:
        break

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
g, y = ds.batch(by=['g', 'y'])
torch.nn.functional.mse_loss(DumbModel(input_)(g).mean, target)

tensor(1.2900)

In [37]:
class DumbModel(object):
    def __init__(self, y):
        self.y = y

    def __call__(self, *args, **kwargs):
        return torch.distributions.Normal(
            self.y.ravel().cuda(), 1.0
        )

import torch
import malt
from malt.metrics.supervised_metrics import MSE, MAPE, RMSE, R2
from malt.metrics.base_metrics import mse, mape, rmse, r2
input_ = torch.randn(5, 1)
target = torch.randn(5, 1)

ds = malt.Dataset()

for idx in range(5):
    point = malt.Molecule(smiles="C", metadata={'y': target[idx].item()})
    ds.append(point)

assert MSE()(DumbModel(input_), ds) == mse(input_, target)
assert MAPE()(DumbModel(input_), ds) == mape(input_, target)
assert RMSE()(DumbModel(input_), ds) == rmse(input_, target)
assert R2()(DumbModel(input_), ds) == r2(input_, target)

c:\users\micha\dev\choderalab\malt\malt\metrics\base_metrics.py:4: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return torch.nn.functional.mse_loss(target, input)


RuntimeError: The size of tensor a (20) must match the size of tensor b (5) at non-singleton dimension 0

In [30]:
ds_te = ds
g, y = ds_te.batch(by=['g', 'y'])
with torch.no_grad():
    y_hat = model(g).mean

getattr(malt.metrics.base_metrics, 'mse')(y_hat, y)

tensor(2.3256, device='cuda:0', grad_fn=<MseLossBackward0>)

In [16]:
mse(input_, target)

tensor(1.6681)

In [15]:
MSE()(DumbModel(input_), ds)

tensor(1.6681, device='cuda:0')

In [ ]:
g, y = ds.batch(by=['g', 'y'])
y_hat = DumbModel(input_)(g).mean
malt.metrics.base_metrics.mse(y_hat, target)

In [12]:
MSE()(DumbModel(input_), ds)

tensor(1.8533, device='cuda:0')

In [5]:
mse(input_, target)

tensor(1.6798)

In [6]:
%load_ext autoreload
%autoreload 2

import malt
import torch
from malt.agents.player import SequentialModelBasedPlayer

dataset = malt.data.collections.linear_alkanes(10)
g, y = dataset.batch()
model = malt.models.supervised_model.GaussianProcessSupervisedModel(
   representation=malt.models.representation.DGLRepresentation(
       out_features=128
   ),
   regressor=malt.models.regressor.HardcodedExactGaussianProcessRegressor(
       y, in_features=128, out_features=2,
   ),
   likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

if torch.cuda.is_available():
    model.cuda()

mll = malt.models.marginal_likelihood.HardcodedExactMarginalLogLikelihood(
    model.likelihood, model
)

# mll = malt.models.marginal_likelihood.ExactMarginalLogLikelihood(
#     model.regressor.likelihood, model
# )

player = SequentialModelBasedPlayer(
   model = model,
   policy=malt.policy.Greedy(),
   trainer=malt.trainer.get_default_trainer(),
   marginal_likelihood=mll,
   merchant=malt.agents.merchant.DatasetMerchant(dataset),
   assayer=malt.agents.assayer.DatasetAssayer(dataset),
)

while True:
    if player.step() is None:
        break

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
player.portfolio.smiles

['CCCC',
 'CCC',
 'CCCCC',
 'CCCCCC',
 'CCCCCCC',
 'CCCCCCCC',
 'CCCCCCCCC',
 'CCCCCCCCCC',
 'CC',
 'C']

In [40]:
import malt
import torch
from malt.agents.player import SequentialModelBasedPlayer

dataset = malt.data.collections.linear_alkanes(10)
g, y = dataset.batch()
model = malt.models.supervised_model.GaussianProcessSupervisedModel(
   representation=malt.models.representation.DGLRepresentation(
       out_features=32
   ),
   regressor=malt.models.regressor.ExactGaussianProcessRegressor(
       y, in_features=32, out_features=2,
   ),
   likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)
if torch.cuda.is_available():
    model.cuda()

mll = malt.models.marginal_likelihood.ExactMarginalLogLikelihood(
    model.regressor.likelihood,
    model
)

player = SequentialModelBasedPlayer(
    model = model,
    policy=malt.policy.Greedy(),
    trainer=malt.trainer.get_default_trainer(),
    marginal_likelihood=mll,
    merchant=malt.agents.merchant.DatasetMerchant(dataset),
    assayer=malt.agents.assayer.DatasetAssayer(dataset),
)

while True:
    if player.step() is None:
        break

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 1 but got size 32 for tensor number 1 in the list.

In [34]:
player.portfolio.smiles

['CCCC',
 'CCCCCC',
 'CCCCCCCC',
 'CCCCCCCCC',
 'CCCCCCCCCC',
 'CC',
 'CCCCC',
 'CCCCCCC',
 'C',
 'CCC']